Coastal change is a global phenomenon that is attributed to tides, powerful sea currents and overall climate change. It has the potential to threaten communities and local economies of coastal towns and cities.

We developed an end-to-end GBDX workflow for coastal change detection and measurement at the native resolution of our 8-band multispectral imagery. This example focuses on Cape Cod, an area which is well known for extreme changes in the coastal landscape. The workflow takes two roughly collocated images of Cape Cod, captured in 2010 and 2016 by WorldView-2 and WorldView-3, respectively, and computes coastal change on the entire images, roughly an area of 1500km2 in less than 30 minutes.

The full story is [here](http://gbdxstories.digitalglobe.com/coastal-change/).

In [ ]:
# Specify your credentials and create a gbdx interface

import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools
gbdx = gbdxtools.Interface()

Run the following workflow if you want to order and preprocess the imagery yourself. We've already done this so this step is optional. In this case, preprocessing involves ortho-rectification, projection to UTM coordinates, and atmospheric compensation, and takes about one hour per image. Note that if the imagery is not already on GBDX, the ordering step can take a few hours.  

In [ ]:
# specify catalog ids
catalog_ids = ['1030010004C45A00', '104001001E9EF700']

# create order task
# it the images are not on GBDX, this task will order them from the DG factory
order = gbdx.Task('Auto_Ordering')
# pass both catalog ids as inputs - this will launch a batch workflow
order.inputs.cat_id = catalog_ids
# for this particular task, we need to set this flag to true
order.impersonation_allowed = True

# create preprocessing task and set input parameters
aop = gbdx.Task('AOP_Strip_Processor')
aop.inputs.data = order.outputs.s3_location.value
aop.inputs.bands = 'MS'
aop.inputs.enable_dra = False
aop.inputs.enable_pansharpen = False
aop.inputs.enable_acomp = True
aop.inputs.enable_tiling = False
aop.inputs.ortho_epsg = 'UTM'     # this setting is optional

# create two-task preprocessing workflow
preprocess_wf = gbdx.Workflow([order, aop])

# set output location to platform-stories/trial-runs/random_str within your bucket/prefix
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

# the two processed images will be stored under output_location
preprocess_wf.savedata(aop.outputs.data, output_location)

# execute
preprocess_wf.execute()

# monitor status
preprocess_wf.status

The first step is image pair alignment.

In [4]:
from os.path import join

# S3 location of the two images
input_location = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/coastal-change/images'

# Image alignment
ipa = gbdx.Task('protogenV2CD_READY')
ipa.inputs.raster = join(input_location, 'post')
ipa.inputs.slave  = join(input_location, 'pre')

Then the water masks are computed.

In [5]:
# Water masks
water_pre = gbdx.Task('protogenV2RAW')
water_post = gbdx.Task('protogenV2RAW')
water_pre.inputs.raster = ipa.outputs.slave.value
water_post.inputs.raster = ipa.outputs.data.value

The exclusion mask removes irrelevant, i.e., non-water-related, change.

In [6]:
# Exclusion mask
# Enable raid domain as this task is compute-intensive
exclusion_mask = gbdx.Task('protogenV2CD_LULC')
exclusion_mask.domain = 'raid'
exclusion_mask.inputs.raster = ipa.outputs.data.value
exclusion_mask.inputs.slave = ipa.outputs.slave.value

Compute the gain, loss and tri-state maps, as well as the discrete distance transforms of the gain and loss maps.

In [7]:
# Water gain map
bcd_gain = gbdx.Task('protogenV2CD_BIN_GAIN')
bcd_gain.inputs.raster = water_post.outputs.data.value
bcd_gain.inputs.slave  = water_pre.outputs.data.value
bcd_gain.inputs.mask   = exclusion_mask.outputs.data.value

# Water loss map
bcd_loss = gbdx.Task('protogenV2CD_BIN_LOSS')
bcd_loss.inputs.raster = water_post.outputs.data.value
bcd_loss.inputs.slave  = water_pre.outputs.data.value
bcd_loss.inputs.mask   = exclusion_mask.outputs.data.value

# Tristate map
bcd_tri = gbdx.Task('protogenV2CD_BIN_TRI')
bcd_tri.inputs.raster = water_post.outputs.data.value
bcd_tri.inputs.slave  = water_pre.outputs.data.value
bcd_tri.inputs.mask   = exclusion_mask.outputs.data.value

# Discrete distance transform of gain and loss maps
ddt_gain = gbdx.Task('protogenV2CD_GDDT')
ddt_gain.inputs.raster = water_post.outputs.data.value
ddt_gain.inputs.slave = bcd_gain.outputs.data.value
ddt_loss = gbdx.Task('protogenV2CD_LDDT')
ddt_loss.inputs.raster = water_post.outputs.data.value
ddt_loss.inputs.slave  = bcd_loss.outputs.data.value

Create the workflow object, specify where to save outputs and execute.

In [8]:
# Define the workflow
wf = gbdx.Workflow([ipa, water_pre, water_post, exclusion_mask, bcd_tri, bcd_loss, bcd_gain, ddt_gain, ddt_loss])

# Define output location
output_location = 'platform-stories/trial-runs/coastal-change'

# Define where to save outputs from various tasks
wf.savedata(water_pre.outputs.data, output_location + '/water_pre')
wf.savedata(water_post.outputs.data, output_location + '/water_post')
wf.savedata(exclusion_mask.outputs.data, output_location + '/exclusion_mask')
wf.savedata(bcd_tri.outputs.data, output_location + '/bcd_tristate')
wf.savedata(bcd_gain.outputs.data, output_location + '/bcd_gain')
wf.savedata(bcd_loss.outputs.data, output_location + '/bcd_loss')
wf.savedata(ddt_gain.outputs.data, output_location + '/ddt_gain')
wf.savedata(ddt_loss.outputs.data, output_location + '/ddt_loss')

# Execute the workflow
wf.execute()

u'4720931332541981473'

Check status.

In [14]:
wf.status

{u'event': u'succeeded', u'state': u'complete'}

Create map.

In [10]:
# Create slippy map
from ipyleaflet import Map, TileLayer

m = Map(center=[41.67, -70.00], zoom=12)

# Mapbox TMS URLs
mapbox_token = 'pk.eyJ1IjoicGxhdGZvcm1zdG9yaWVzIiwiYSI6ImNpeTZkeWlvOTAwNm0yeHFocHFyaGFleDcifQ.wOsbVsS0NXKrWeX2bQwc-g'
url_pre = 'https://a.tiles.mapbox.com/v4/platformstories.coastal-change-pre/{z}/{x}/{y}.png?access_token=' + mapbox_token
url_tristate = 'https://a.tiles.mapbox.com/v4/platformstories.coastal-change-tristate/{z}/{x}/{y}.png?access_token=' + mapbox_token

m.add_layer(TileLayer(url=url_pre))
m.add_layer(TileLayer(url=url_tristate, opacity=0.5))
    
# launch map    
m